In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd

query = "SELECT * FROM disclosure_events"
df = pd.read_sql(query, engine)
print(f"데이터베이스에서 조회된 공시 이벤트 수: {len(df)}")
df.head()


DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


데이터베이스에서 조회된 공시 이벤트 수: 31165


,id,stock_code,market,company_name,report_name,disclosed_at,summary_kr,raw,created_at,ret_1m,ret_3m,ret_10m,ret_15m,ret_30m,ret_60m,label,score
0,32519,KR7114190002,KOSDAQ,강원에너지,타법인주식및출자증권취득결정(자율공시),2022-07-01 01:03:17+00:00,"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...","2022.07.01 10:03:16\n기업명: 강원에너지(시가총액: 1,842억)\...",2025-08-09 19:13:15.234304,-0.14,0.00,-1.99,-3.55,-3.83,-3.69,neutral,0.9723
1,52054,KR7068270008,KOSPI,셀트리온,투자판단관련주요경영사항(CT-P42(아일리아 바이오시밀러) 임상 3상 결과 발표),2023-04-02 23:00:06+00:00,셀트리온은 당뇨병성 황반부종 환자를 대상으로 한 아일리아 바이오시밀러(CP-T42)...,2023.04.03 08:00:05\n기업명: 셀트리온(시가총액: 21조 948억)...,2025-08-09 19:13:27.639981,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,33989,None,None,CBI,주요사항보고서(무상증자결정),2022-07-29 07:26:53+00:00,"CBI는 2022년 7월 29일 무상증자를 결정하였으며, 발행일은 9월 6일이고 기...","2022.07.29 16:26:52\n기업명: CBI(시가총액: 1,681억)\n보...",2025-08-09 19:13:15.234304,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,32520,KR7082920000,KOSDAQ,비츠로셀,단일판매ㆍ공급계약체결,2022-07-01 01:24:45+00:00,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,"2022.07.01 10:24:44\n기업명: 비츠로셀(시가총액: 2,477억)\n...",2025-08-09 19:13:15.234304,-0.85,0.43,0.00,1.28,0.85,0.85,positive,0.9959
4,64741,KR7381970003,KOSPI,케이카,영업(잠정)실적(공정공시),2023-11-01 23:57:28+00:00,"케이카는 2023년 3분기 매출액 5,286억, 영업이익 184억, 순이익 54억을...","2023.11.02 08:57:27\n기업명: 케이카(시가총액: 4,727억)\n보...",2025-07-08 17:09:01.716527,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [3]:
# market 종류와 각각의 비율 분석
market_counts = df['market'].value_counts()
print("시장별 공시 분포:")
print(market_counts)
print(f"\n총 공시 건수: {len(df)}")
print("\n시장별 비율:")
for market, count in market_counts.items():
    ratio = count / len(df) * 100
    print(f"{market}: {count}건 ({ratio:.1f}%)")


시장별 공시 분포:
market
KOSDAQ           13403
KOSPI            12392
KOSDAQ GLOBAL     1057
Name: count, dtype: int64

총 공시 건수: 31165

시장별 비율:
KOSDAQ: 13403건 (43.0%)
KOSPI: 12392건 (39.8%)
KOSDAQ GLOBAL: 1057건 (3.4%)


In [4]:
# disclosed_at을 datetime으로 변환 (이미 timezone-aware 상태)
df['disclosed_at'] = pd.to_datetime(df['disclosed_at'])

# 이미 timezone-aware이므로 tz_convert만 사용하여 KST로 변환
df['disclosed_at_kst'] = df['disclosed_at'].dt.tz_convert('Asia/Seoul')

# KST 시간에 따라 데이터프레임 정렬
df = df.sort_values('disclosed_at_kst')

# 시간만 추출 (KST 기준)
df['time_only'] = df['disclosed_at_kst'].dt.time



# 장중 시간 정의 (9:00 ~ 15:30)
import datetime
market_open = datetime.time(9, 0)
market_close = datetime.time(15, 30)

# 장중/장외 분류
df['market_session'] = df['time_only'].apply(
    lambda x: '장중' if market_open <= x <= market_close else '장외'
)

# 결과 집계
session_counts = df['market_session'].value_counts()
print("공시 발표 시간대별 분포:")
print(session_counts)
print(f"\n장중 비율: {session_counts['장중'] / len(df) * 100:.1f}%")
print(f"장외 비율: {session_counts['장외'] / len(df) * 100:.1f}%")


공시 발표 시간대별 분포:
market_session
장외    19745
장중    11420
Name: count, dtype: int64

장중 비율: 36.6%
장외 비율: 63.4%


In [7]:
import pandas as pd
from datetime import datetime

# 날짜 범위 설정
start_date = datetime(2022, 7, 1)
end_date = datetime(2023, 12, 28)

table = "disclosure_events"
query = f"""SELECT * FROM {table}
WHERE stock_code is not NULL 
  AND market IN ('KOSPI', 'KOSDAQ', 'KOSDAQ GLOBAL') 
  AND (
      (EXTRACT(HOUR FROM disclosed_at AT TIME ZONE 'Asia/Seoul') BETWEEN 9 AND 14)
      OR
      (EXTRACT(HOUR FROM disclosed_at AT TIME ZONE 'Asia/Seoul') = 15
       AND EXTRACT(MINUTE FROM disclosed_at AT TIME ZONE 'Asia/Seoul') <= 30)
  )
  AND disclosed_at >= %(start_date)s
  AND disclosed_at <= %(end_date)s
ORDER BY id ASC"""

target_df = pd.read_sql(query, engine, params={"start_date": start_date, "end_date": end_date})

target_df

,id,stock_code,market,company_name,report_name,disclosed_at,summary_kr,raw,created_at,ret_1m,ret_3m,ret_10m,ret_15m,ret_30m,ret_60m,label,score
0,32519,KR7114190002,KOSDAQ,강원에너지,타법인주식및출자증권취득결정(자율공시),2022-07-01 01:03:17+00:00,"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...","2022.07.01 10:03:16\n기업명: 강원에너지(시가총액: 1,842억)\...",2025-08-09 19:13:15.234304,-0.14,0.00,-1.99,-3.55,-3.83,-3.69,neutral,0.9723
1,32520,KR7082920000,KOSDAQ,비츠로셀,단일판매ㆍ공급계약체결,2022-07-01 01:24:45+00:00,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,"2022.07.01 10:24:44\n기업명: 비츠로셀(시가총액: 2,477억)\n...",2025-08-09 19:13:15.234304,-0.85,0.43,0.00,1.28,0.85,0.85,positive,0.9959
2,32521,KR7042370007,KOSDAQ,비츠로테크,단일판매ㆍ공급계약체결(자회사의 주요경영사항),2022-07-01 01:27:02+00:00,비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,"2022.07.01 10:27:02\n기업명: 비츠로테크(시가총액: 1,821억)\...",2025-08-09 19:13:15.234304,0.00,-0.14,-0.29,0.14,0.72,1.01,positive,0.9999
3,32522,KR7014820005,KOSPI,동원시스템즈,신규시설투자등(자율공시),2022-07-01 01:43:15+00:00,동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,"2022.07.01 10:43:14\n기업명: 동원시스템즈(시가총액: 1조 5,61...",2025-08-09 19:13:15.234304,0.00,0.18,0.37,0.55,0.74,0.00,neutral,0.6471
4,32524,KR7226950004,KOSDAQ,올릭스,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,2022-07-01 02:30:09+00:00,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,"2022.07.01 11:30:08\n기업명: 올릭스(시가총액: 2,946억)\n보...",2025-08-09 19:13:15.234304,-1.60,-1.37,-0.68,-1.37,-2.28,-2.51,positive,0.9998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9647,68075,KR7445090004,KOSDAQ,에이직랜드,주요사항보고서(회사합병결정),2023-12-27 05:31:32+00:00,에이직랜드는 반도체 디자인 서비스를 제공하는 자회사 탑에이직을 흡수합병하여 사업 통...,"2023.12.27 14:31:31\n기업명: 에이직랜드(시가총액: 6,396억)\...",2025-07-08 17:09:03.053194,0.33,0.17,-0.17,-0.67,-0.50,NaN,neutral,0.6333
9648,68076,KR7445090004,KOSDAQ,에이직랜드,주요사항보고서(회사합병결정),2023-12-27 05:33:57+00:00,에이직랜드는 반도체 디자인 서비스를 주력으로 하는 자회사 위즈마인드를 흡수합병하여 ...,"2023.12.27 14:33:56\n기업명: 에이직랜드(시가총액: 6,428억)\...",2025-07-08 17:09:03.053194,0.17,0.00,-0.17,-0.33,-0.50,NaN,positive,0.8485
9649,68077,KR7018310003,KOSDAQ,삼목에스폼,소송등의제기ㆍ신청(경영권분쟁소송),2023-12-27 05:48:48+00:00,삼목에스폼은 수원지방법원 평택지원에 장부 열람허용 가처분 신청 사건과 관련하여 영업...,"2023.12.27 14:48:47\n기업명: 삼목에스폼(시가총액: 2,436억)\...",2025-07-08 17:09:03.053194,-1.15,-0.12,-0.18,0.18,1.15,NaN,negative,0.9982
9650,68078,KR7012700001,KOSDAQ,리드코프,타법인주식및출자증권처분결정,2023-12-27 06:03:00+00:00,리드코프는 2024년 3월 31일까지 채권추심전문회사인 엘씨대부의 지분 11.71%...,"2023.12.27 15:02:59\n기업명: 리드코프(시가총액: 1,584억)\n...",2025-07-08 17:09:03.053194,0.00,-0.17,-0.17,0.00,NaN,NaN,neutral,0.9999
